# Data Cleaning for VF Ghana — IDP MVP Notebook (Enrichment-Only)

In [24]:
# pip install pandas

In [7]:
# !pip install numpy

In [15]:
# # Importing
# import ast, re
# import pandas as pd
# import numpy as np
# import plotly.express as px

# # Paths (adjust if your CSV is elsewhere)
# DATA_PATH = "Virtue Foundation Ghana v0.3 - Sheet1.csv"

In [ ]:
import re, ast
import pandas as pd


# Reading data
DATA_PATH = "Virtue Foundation Ghana v0.3 - Sheet1.csv"  # your CSV
df_raw = pd.read_csv(DATA_PATH)     # keep raw intact
df = df_raw.copy()                  # only add columns to this copy

print(f"Rows: {len(df):,} | Cols: {len(df.columns)}")

Rows: 987 | Cols: 41


In [ ]:
# Normalize common “null tokens” → shadow columns

# Create shadow columns that turn "null" | "N/A" | "[]" | "" into Python None. Originals stay as‑is.

NULL_TOKENS = {"", "null", "none", "n/a", "na", "[]", "{}", " ", "\t", "-"}

def to_none_if_placeholder(x):
    if x is None:
        return None
    s = str(x).strip()
    return None if s.lower() in NULL_TOKENS else x

# Pick columns where placeholders often show up
maybe_nullish_cols = [
    "phone_numbers", "email", "websites", "officialWebsite",
    "address_line1","address_line2","address_line3",
    "address_city","address_stateOrRegion","address_zipOrPostcode",
    "address_country","address_countryCode",
    "specialties","procedure","equipment","capability","description"
]

for col in maybe_nullish_cols:
    if col in df.columns:
        df[f"{col}_clean"] = df[col].apply(to_none_if_placeholder)

df.filter(regex="_clean$").head(3)

,phone_numbers_clean,email_clean,websites_clean,officialWebsite_clean,address_line1_clean,address_line2_clean,address_line3_clean,address_city_clean,address_stateOrRegion_clean,address_zipOrPostcode_clean,address_country_clean,address_countryCode_clean,specialties_clean,procedure_clean,equipment_clean,capability_clean,description_clean
0,"[""+233249354576"",""+233203928883""]",NaN,"[""waafweb.org""]",waafweb.org,109/No 1 Bekwai Rd (Near Mexico Hotel),NaN,NaN,Takoradi,NaN,NaN,Ghana,GH,"[""infectiousDiseases"",""maternalFetalMedicineOr...",NaN,NaN,"[""Has a location at 109/No 1 Bekwai Rd (Near M...","WAAF is committed to battling HIV/AIDS, TB, an..."
1,NaN,NaN,NaN,NaN,Opp. Standard Chartered Bank,NaN,NaN,Dansoman,NaN,NaN,Ghana,GH,"[""internalMedicine""]",NaN,NaN,"[""Located in Dansoman, Accra, Ghana, opposite ...",NaN
2,NaN,NaN,NaN,NaN,"Opp. Standard Chartered Bank, Dansoman",NaN,NaN,Accra,NaN,NaN,Ghana,GH,"[""internalMedicine""]",NaN,NaN,NaN,NaN


In [ ]:
# Parse list‑like text into *_list (schema‑friendly, non‑destructive)

# Before
# specialties = ["cardiology", "pediatrics"]
# After
# specialties_list = ["cardiology", "pediatric"]

LIST_LIKE_COLS = ["specialties","procedure","equipment","capability",
                  "phone_numbers","websites","countries"]

def parse_listlike(val):
    if pd.isna(val):
        return None
    if isinstance(val, list):
        out = [str(x).strip() for x in val if str(x).strip()]
        return out or None
    s = str(val).strip()
    if not s:
        return None
    # Try to decode '["a","b"]'
    try:
        obj = ast.literal_eval(s)
        if isinstance(obj, (list, tuple)):
            out = [str(x).strip() for x in obj if str(x).strip()]
            return out or None
    except Exception:
        pass
    # Fallback: split by ';' or '|'
    if ";" in s or "|" in s:
        parts = re.split(r"[;|]", s)
        parts = [p.strip() for p in parts if p.strip()]
        return parts or None
    # Else single-item list
    return [s]

for col in LIST_LIKE_COLS:
    if col in df.columns:
        df[col + "_list"] = df[col].apply(parse_listlike)

df[["specialties","specialties_list","procedure","procedure_list"]].head(4)

,specialties,specialties_list,procedure,procedure_list
0,"[""infectiousDiseases"",""maternalFetalMedicineOr...","[infectiousDiseases, maternalFetalMedicineOrPe...",NaN,None
1,"[""internalMedicine""]",[internalMedicine],[],None
2,"[""internalMedicine""]",[internalMedicine],[],None
3,"[""internalMedicine""]",[internalMedicine],[],None


In [ ]:
# Light address normalization → add region_norm, country_norm

# Adds normalized helpers for region/country without changing original address fields. Useful for regional synthesis later

REGION_NORMALIZE = {
    "ashanti":"Ashanti","greater accra":"Greater Accra","western":"Western","western region":"Western",
    "western north":"Western North","eastern":"Eastern","central":"Central","volta":"Volta","oti":"Oti",
    "savannah":"Savannah","north east":"North East","northeast":"North East","northern":"Northern",
    "upper east":"Upper East","upper west":"Upper West","bono":"Bono","bono east":"Bono East","ahafo":"Ahafo",
}

def normalize_region(x):
    if pd.isna(x): return None
    s = str(x).strip()
    key = s.lower().replace("region","").strip()
    return REGION_NORMALIZE.get(key, s)

df["region_norm"]  = df.get("address_stateOrRegion", pd.Series([None]*len(df))).apply(normalize_region)
df["country_norm"] = df.get("address_country", pd.Series([None]*len(df))).apply(lambda x: None if pd.isna(x) else str(x).strip())

df[["address_stateOrRegion","region_norm","address_country","country_norm"]].head(6)

,address_stateOrRegion,region_norm,address_country,country_norm
0,NaN,NaN,Ghana,Ghana
1,NaN,NaN,Ghana,Ghana
2,NaN,NaN,Ghana,Ghana
3,NaN,NaN,Ghana,Ghana
4,NaN,NaN,Ghana,Ghana
5,Western,Western,Ghana,Ghana


In [ ]:
# Gentle web/contact tidying → shadow *_clean
# We only standardize obviously safe things: strip spaces, lower‑case domains; do not remove or rewrite values.

"""
Like -

https://www.example.org/
http://example.org
example.org
NULL

All are same but looking differnt, so making all in commong www.example.org format
"""

def clean_email(x):
    if pd.isna(x): return None
    s = str(x).strip()
    # keep as-is if malformed; LLM might still use it as evidence
    return s.lower()

def clean_domain(x):
    if pd.isna(x): return None
    s = str(x).strip()
    # remove protocol if present, keep host/path
    s = re.sub(r"^https?://", "", s, flags=re.I)
    return s.rstrip("/")

if "email_clean" not in df.columns and "email" in df.columns:
    df["email_clean"] = df["email"].apply(clean_email)

if "officialWebsite" in df.columns:
    df["officialWebsite_clean"] = df["officialWebsite"].apply(clean_domain)

if "websites_list" in df.columns:
    df["websites_clean"] = df["websites_list"].apply(lambda L: [clean_domain(x) for x in L] if isinstance(L, list) else None)

df[["email","email_clean","officialWebsite","officialWebsite_clean"]].head(6)

,email,email_clean,officialWebsite,officialWebsite_clean
0,NaN,NaN,waafweb.org,waafweb.org
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN


In [ ]:
# Schema‑aligned facility type helper (non‑destructive)

# Adds a coarse, human‑friendly type for filters and tables (hospital/clinic/maternity/dentist/diagnostic/ngo/other) without modifying schema fields.
# Handy for filtering and summarizing facilities in UI or pivot tables; still non‑destructive.

FACILITY_TYPE_CANON = {"hospital","clinic","maternity","diagnostic","pharmacy","dentist","ngo"}

def facility_type_simple(row):
    # Use the schema fields first if present
    for col in ("facilityTypeId", "organization_type"):
        if col in df.columns and pd.notna(row.get(col)):
            s = str(row[col]).lower()
            for t in FACILITY_TYPE_CANON:
                if t in s:
                    return t
    # Fallback from name
    name = str(row.get("name","")).lower()
    if "hospital" in name: return "hospital"
    if "clinic"   in name: return "clinic"
    if "maternity" in name: return "maternity"
    if "dental" in name or "dentist" in name: return "dentist"
    return "other"

df["facility_type_simple"] = df.apply(facility_type_simple, axis=1)
df[["name","facilityTypeId","organization_type","facility_type_simple"]].head(8)

,name,facilityTypeId,organization_type,facility_type_simple
0,109/No 1 Bekwai Rd (Near Mexico Hotel) Takorad...,clinic,facility,clinic
1,1st Foundation Clinic,clinic,facility,clinic
2,1st Foundation Clinic,clinic,facility,clinic
3,1st Foundation Clinic,clinic,facility,clinic
4,1st Foundation Clinic,clinic,facility,clinic
5,2BN Military Hospital,hospital,facility,hospital
6,37 Military Hospital,hospital,facility,hospital
7,37 Military Hospital,hospital,facility,hospital


In [ ]:
# (Optional) Create a light evidence blob for later LLM/rules

# This just concatenates a few free‑text fields so your regex/LLM has a single place to look—no changes to originals.

SEARCH_COLS = ["specialties","procedure","equipment","capability","description"]

def make_blob(row):
    parts = []
    for c in SEARCH_COLS:
        if c in df.columns and pd.notna(row.get(c)) and str(row[c]).strip():
            parts.append(str(row[c]))
    return "\n".join(parts)

df["_blob"] = df.apply(make_blob, axis=1)
df[["_blob"]].head(3)


,_blob
0,"[""infectiousDiseases"",""maternalFetalMedicineOr..."
1,"[""internalMedicine""]\n[]\n[]\n[""Located in Dan..."
2,"[""internalMedicine""]\n[]\n[]\n[]"


In [30]:
print("column", len(df.columns))
print("row", len(df))
# list(df.columns)

column 69
row 987


In [32]:
# Save an enriched view

df.to_csv("vf_ghana_enriched.csv", index=False)

#### Only‑added helper columns in df:
```
*_clean (emails, websites, address fields)
*_list (schema’s list fields parsed non‑destructively)
region_norm, country_norm (for regional synthesis)
facility_type_simple (for simple grouping)
_blob (optional convenience for extraction)
```